In [ ]:
from pyspark import SparkContext, SparkConf
from fnvhash import fnv1a_32

#Configure pyspark
conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf=conf)

#Create rdds for documents
document_1 = sc.textFile("titanic.csv")
document_2 = sc.textFile("titanic.csv")

#Set the value k (the length of the shingles); k-shingles
k = 2

In [ ]:

#Method that takes an rdd and returns an rdd of hashed shingles.
def Shingling(document):
    shingles = []
    for i in range(0,len(document)-k+1):
        currentShingle = document[i:i+k]
        #Convert shingles to integer representation
        hashed_shingle = fnv1a_32(str.encode(currentShingle))
        #Append hashed value to list
        shingles.append(hashed_shingle)
    return set(shingles)

#Method that takes two rdds and computes the Jaccard similarity
def CompareSets(setA,setB):
    union = sc.union([setA, setB]).count()
    intersection = setA.intersection(setB).count()
    return intersection/union


In [ ]:
#Create hashed shinglings from documents
rdd_shingles_document1 = document_1.map(Shingling).flatMap(lambda list: list).distinct()
rdd_shingles_document2 = document_2.map(Shingling).flatMap(lambda list: list).distinct()

#Get the Jaccard similarity
CompareSets(rdd_shingles_document1,rdd_shingles_document2)

In [ ]:
#union = sc.union([shingles_document1, shingles_document2]).distinct()
hashedShingles1 = document_1.map(Shingling).flatMap(lambda list: list).collect()
hashedShingles2 = document_2.map(Shingling).flatMap(lambda list: list).collect()
def createMatrix(shingle):
    if shingle in hashedShingles1 and shingle in hashedShingles2:
        return (shingle,1,1)
    if shingle in hashedShingles1 and shingle not in hashedShingles2:
        return (shingle,1,0)
    if shingle not in hashedShingles1 and shingle in hashedShingles2:
        return (shingle,0,1)
    else:
        return (shingle,0,0)

matrix = sc.union([rdd_shingles_document1, rdd_shingles_document2]).distinct().map(createMatrix)

def removeZeros(s):
    if s[1] == 1 or s[2] == 1:
        return s
    else:
        return
matrix = matrix.filter(lambda x: x[2] == 1 or x[1] == 1).collect()
matrix

In [ ]:
#def CompareSignatures(matrix):
    
#    return